# Coding etiquete and excel reporting, Part 1, step 5

## Content list:

#### 01. Importing libraries¶
#### 02. Importing data
#### 03. Profiles of customers (question 5)
##### 03.1 Age flag
##### 03.2 Income flag
##### 03.3 Family flag
##### 03.4 Food preference profile
###### 03.4.1 Exploring with crosstabs and pivot tables to compare results
##### 03.5 Popular departments flag
##### 03.6 Popular departments and family size
#### 04. Exporting dataframe (normal_cust_prods)

## 01. Importing libraries

In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

## 0.2 Importing data

In [2]:
path= r'C:\Users\isobr\OneDrive\02122022Instacart Basket Analysis'

In [3]:
path

'C:\\Users\\isobr\\OneDrive\\02122022Instacart Basket Analysis'

In [4]:
cust_prof = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared data', 'normal_activity_cust.pkl'))

In [5]:
cust_prof.head(3)

,Unnamed: 0_x,product_id,product_name,aisle_id,department_id,prices,Unnamed: 0.1,Unnamed: 0_y,order_id,user_id,...,surname,Gender,State,Age,date_joined,number_dependants,family_status,income,region,activity_flag
0,0,1,Chocolate Sandwich Cookies,61,19,5.8,1987,1987,3139998,138,...,Cox,Male,Minnesota,81,2019-08-01,1,married,49620,Midwest,Normal activity
1,0,1,Chocolate Sandwich Cookies,61,19,5.8,1989,1989,1977647,138,...,Cox,Male,Minnesota,81,2019-08-01,1,married,49620,Midwest,Normal activity
2,907,907,Premium Sliced Bacon,106,12,20.0,1960,1960,3160996,138,...,Cox,Male,Minnesota,81,2019-08-01,1,married,49620,Midwest,Normal activity


In [6]:
# department dataset
df_dep = pd.read_csv(os.path.join(path, '02 Data', 'Prepared data', 'departments_wrangled.csv'))

In [7]:
#checking the number of departments and the products typology
df_dep

,Unnamed: 0,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol
5,6,international
6,7,beverages
7,8,pets
8,9,dry goods pasta
9,10,bulk


## 0.3 Profiles of customers

#### 0.3.1 Age flag

First profile to be created is an age flag, using min. and max age limits of clients

In [8]:
cust_prof['Age'].min()

18

In [9]:
cust_prof['Age'].max()

81

In [10]:
# profile customers by age group
cust_prof.loc[(cust_prof['Age']>=18) & (cust_prof['Age']<=35),'age_flag']='Young Adult'

In [11]:
cust_prof.loc[(cust_prof['Age']>=35)&(cust_prof['Age']<=50),'age_flag']='Middle-Aged'

In [12]:
cust_prof.loc[(cust_prof['Age']>=51)&(cust_prof['Age']<=65),'age_flag']='Older Adult'

In [13]:
cust_prof.loc[cust_prof['Age']>65,'age_flag']='Senior'

In [14]:
cust_prof.head(2)

,Unnamed: 0_x,product_id,product_name,aisle_id,department_id,prices,Unnamed: 0.1,Unnamed: 0_y,order_id,user_id,...,Gender,State,Age,date_joined,number_dependants,family_status,income,region,activity_flag,age_flag
0,0,1,Chocolate Sandwich Cookies,61,19,5.8,1987,1987,3139998,138,...,Male,Minnesota,81,2019-08-01,1,married,49620,Midwest,Normal activity,Senior
1,0,1,Chocolate Sandwich Cookies,61,19,5.8,1989,1989,1977647,138,...,Male,Minnesota,81,2019-08-01,1,married,49620,Midwest,Normal activity,Senior


In [15]:
#checking the value_counts
cust_prof['age_flag'].value_counts(dropna=False)

Young Adult    8242295
Middle-Aged    7774770
Senior         7730834
Older Adult    7216665
Name: age_flag, dtype: int64

Young adults have the highest frequency

#### 03.2 Income flag

Second variable to be created will group clients by income type

In [16]:
#First, looking into statistics for income
cust_prof['income'].describe()

count    3.096456e+07
mean     9.967587e+04
std      4.314187e+04
min      2.590300e+04
25%      6.729200e+04
50%      9.676500e+04
75%      1.281020e+05
max      5.939010e+05
Name: income, dtype: float64

In [17]:
cust_prof['income'].max()

593901

In [18]:
cust_prof['income'].min()

25903

It is difficult to make a good division based on the data alone, so I used the following reference for income groups: 

##### Poor or near-poor  32,048𝑜𝑟 𝑙𝑒𝑠𝑠𝐿𝑜𝑤𝑒𝑟−𝑚𝑖𝑑𝑑𝑙𝑒𝑐𝑙𝑎𝑠𝑠 
##### 32,048 -  53,413 𝑀𝑖𝑑𝑑𝑙𝑒𝑐𝑙𝑎𝑠𝑠 
##### 53,413 -  106,827𝑈𝑝𝑝𝑒𝑟−𝑚𝑖𝑑𝑑𝑙𝑒𝑐𝑙𝑎𝑠𝑠
##### 106,827 -  373,894 𝑅𝑖𝑐ℎ 
##### 373,894 and up are super-rich
###### Link: https://money.usnews.com/money/personal-finance/family-finance/articles/where-do-i-fall-in-the-american-economic-class-system

In [19]:
#Creating a loc function to produce the income flag
cust_prof.loc[cust_prof['income']<30000,'income_flag']='Low Income'

In [20]:
cust_prof.loc[(cust_prof['income']>=30000)&(cust_prof['income']<54000),'income_flag']='Low Middle Income'

In [21]:
cust_prof.loc[(cust_prof['income']>=54000)&(cust_prof['income']<107000),'income_flag']='Middle Income'

In [22]:
cust_prof.loc[(cust_prof['income']>=107000)&(cust_prof['income']<374000),'income_flag']='Upper Income'

In [23]:
cust_prof.loc[cust_prof['income']>=374000,'income_flag']='High Income'

In [24]:
#checking the value counts
cust_prof['income_flag'].value_counts(dropna=False)

Middle Income        14470856
Upper Income         12246474
Low Middle Income     3995310
Low Income             193870
High Income             58054
Name: income_flag, dtype: int64

Most customers are from the middle and upper income. Higher income make up the smallest group, followed by low income. 

#### 03.3 Family flag

Third variable created is a family flag, based on the number od dependants

In [25]:
# First checking max, mean and min values for number of dependants: 
cust_prof['number_dependants'].max()

3

In [26]:
cust_prof['number_dependants'].min()

0

In [27]:
cust_prof['number_dependants'].mean()

1.501819176268718

Based on this information we can have three groups:  no dependants, 1 or 2 dependants and 3 dependants (we can put this one with 3 or more in case new customers appear with more than 3)

In [28]:
# creating the loc function the dependants flag, which I will call family flag
cust_prof.loc[cust_prof['number_dependants'] >= 0 ,'family_flag']='Small Family'

In [29]:
cust_prof.loc[(cust_prof['number_dependants'] >= 1) & (cust_prof['number_dependants'] <= 2),'family_flag']='Middle-Size Family'

In [30]:
cust_prof.loc[cust_prof['number_dependants']>=3,'family_flag']='Large Family'

In [31]:
cust_prof['family_flag'].value_counts(dropna=False)

Middle-Size Family    15452367
Large Family           7772516
Small Family           7739681
Name: family_flag, dtype: int64

The majority of customers are middle-size families with one or two dependants, small families make up the smallest share of customers, yet closely followed by large families.

In [32]:
cust_prof.head()

,Unnamed: 0_x,product_id,product_name,aisle_id,department_id,prices,Unnamed: 0.1,Unnamed: 0_y,order_id,user_id,...,Age,date_joined,number_dependants,family_status,income,region,activity_flag,age_flag,income_flag,family_flag
0,0,1,Chocolate Sandwich Cookies,61,19,5.8,1987,1987,3139998,138,...,81,2019-08-01,1,married,49620,Midwest,Normal activity,Senior,Low Middle Income,Middle-Size Family
1,0,1,Chocolate Sandwich Cookies,61,19,5.8,1989,1989,1977647,138,...,81,2019-08-01,1,married,49620,Midwest,Normal activity,Senior,Low Middle Income,Middle-Size Family
2,907,907,Premium Sliced Bacon,106,12,20.0,1960,1960,3160996,138,...,81,2019-08-01,1,married,49620,Midwest,Normal activity,Senior,Low Middle Income,Middle-Size Family
3,907,907,Premium Sliced Bacon,106,12,20.0,1969,1969,2254091,138,...,81,2019-08-01,1,married,49620,Midwest,Normal activity,Senior,Low Middle Income,Middle-Size Family
4,1000,1000,Apricots,18,10,12.9,1968,1968,505689,138,...,81,2019-08-01,1,married,49620,Midwest,Normal activity,Senior,Low Middle Income,Middle-Size Family


#### 03.4 Food preference profile

Fourth variable will explore the departement to group clients in terms of their preferences

Perishable are bakery (3), produce (4), meat seafood (12), breakfast (14), dairy eggs (16), deli (20)

Non-perishable are frozen (1), beverages (7), dry goods pasta (9), canned goods (15), snacks (19) and alcool (5)

Other: missing, bulk, international because they likely contain both types of foods and also other non-food items.

In [33]:
#First I will create a list for perishable and non-perishable foods 
perishable=[3,4,12,14,16,20]

In [34]:
non_perishable=[1,5,7,9,15,19]

In [35]:
#Now creating a new "for loop" statement
result=[]
for value in cust_prof['department_id']:
  if value in perishable:
    result.append('Perishable Foods')
  elif value in non_perishable:
    result.append('Non-Perishable Foods')
  else:
    result.append('Other products')

In [36]:
#checking if it worked
result

['Non-Perishable Foods',
 'Non-Perishable Foods',
 'Perishable Foods',
 'Perishable Foods',
 'Other products',
 'Perishable Foods',
 'Non-Perishable Foods',
 'Perishable Foods',
 'Perishable Foods',
 'Perishable Foods',
 'Other products',
 'Perishable Foods',
 'Perishable Foods',
 'Perishable Foods',
 'Perishable Foods',
 'Perishable Foods',
 'Perishable Foods',
 'Perishable Foods',
 'Perishable Foods',
 'Perishable Foods',
 'Perishable Foods',
 'Perishable Foods',
 'Perishable Foods',
 'Perishable Foods',
 'Perishable Foods',
 'Perishable Foods',
 'Perishable Foods',
 'Perishable Foods',
 'Perishable Foods',
 'Perishable Foods',
 'Non-Perishable Foods',
 'Perishable Foods',
 'Perishable Foods',
 'Perishable Foods',
 'Perishable Foods',
 'Perishable Foods',
 'Perishable Foods',
 'Perishable Foods',
 'Perishable Foods',
 'Perishable Foods',
 'Perishable Foods',
 'Non-Perishable Foods',
 'Non-Perishable Foods',
 'Non-Perishable Foods',
 'Perishable Foods',
 'Perishable Foods',
 'Perishab

In [37]:
#Now creating a new colummn for product type
cust_prof['product_type']=result

In [38]:
#checking frequency
cust_prof['product_type'].value_counts (dropna=False)

Perishable Foods        17726748
Non-Perishable Foods     9438875
Other products           3798941
Name: product_type, dtype: int64

Most products sold are perishable foods

##### 03.4.1 Exploring with crosstabs and pivot tables to compare results

In [39]:
#now comparing product types and family_flag
crosstab_product = pd.crosstab(cust_prof['product_type'], cust_prof['family_flag'], dropna = False)

In [40]:
crosstab_product

family_flag,Large Family,Middle-Size Family,Small Family
product_type,,,
Non-Perishable Foods,2366204,4715401,2357270
Other products,951170,1902264,945507
Perishable Foods,4455142,8834702,4436904


In [41]:
 crosstab_product.mean()

family_flag
Large Family          2.590839e+06
Middle-Size Family    5.150789e+06
Small Family          2.579894e+06
dtype: float64

In [42]:
#do the same using the pivot table code (taken from Pandas documentation)
pivot = np.round(pd.pivot_table(cust_prof, values='income', 
                                index='product_type', 
                                columns='family_flag', 
                                aggfunc=np.mean),2)

In [43]:
pivot

family_flag,Large Family,Middle-Size Family,Small Family
product_type,,,
Non-Perishable Foods,97651.84,97505.27,97842.75
Other products,101164.04,100674.01,101310.51
Perishable Foods,100440.02,100422.45,100686.83


All family types buy more perishable products, yet non-perishable food sales are slighlty higher for single adults. 

#### 03.5 Popular departments flag

Guided by the frequency of the department_id, it is possible to group departments by popularity

In [44]:
cust_prof['department_id'].value_counts(dropna=False)

4     9079273
16    5177182
19    2766406
7     2571901
1     2121731
13    1782705
3     1120828
15    1012074
20    1003834
9      822136
17     699857
12     674781
14     670850
11     424306
18     410392
6      255991
5      144627
8       93060
21      64768
2       34411
10      33451
Name: department_id, dtype: int64

Top 5 departments are produce (4), dairy_eggs (16), snacks (19), beverages (7) and frozen (1)
The departments with less sales are (10), (2), (21), (8) and (5)
Now I can create a list for the most popular and medium popular and least popular departments

In [45]:
#List for most popular departments
popular=[1,4,7,16,19]

In [46]:
#List for least popular departments
least_popular=[2,5,8,10,21]

In [47]:
#List for other products
regularly_popular=[3,6,9,11,12,13,14,15,17,18,20]

In [48]:
#Now creating a popular department 'flag'
result=[]
for value in cust_prof['department_id']:
  if value in popular:
    result.append('Popular - produce, eggs, dairy, snacks, drinks and frozen')
  elif value in least_popular:
    result.append('Non-popular - bulk, other, missing, pets, alcohol')
  else:
    result.append('Regular - others')

In [49]:
cust_prof['popular_products']=result

In [50]:
cust_prof['popular_products'].value_counts(dropna=False)

Popular - produce, eggs, dairy, snacks, drinks and frozen    21716493
Regular - others                                              8877754
Non-popular - bulk, other, missing, pets, alcohol              370317
Name: popular_products, dtype: int64

In [51]:
#crosstab of department_popularity and family_flag
crosstab_department = pd.crosstab(cust_prof['popular_products'], cust_prof['family_flag'], dropna = False)

In [52]:
crosstab_department

family_flag,Large Family,Middle-Size Family,Small Family
popular_products,,,
"Non-popular - bulk, other, missing, pets, alcohol",93690,183549,93078
"Popular - produce, eggs, dairy, snacks, drinks and frozen",5449117,10832432,5434944
Regular - others,2229709,4436386,2211659


In [53]:
crosstab_department.to_clipboard()

#### 03.6 Popular departments and family size

Now using the lists created for popular departments we can produce a new variable that groups popularity in terms of family size, namely families with dependants and those with no dependants

In [54]:
# Using the lists created above, produce a loc function that organizes popularity of deparments in relation to families
cust_prof.loc[(cust_prof['department_id'].isin(popular))&(cust_prof['number_dependants']>0),'popular_family_flag']='family popular - fresh and frozen'

In [55]:
cust_prof.loc[(cust_prof['department_id'].isin(least_popular))&(cust_prof['number_dependants']>0),'popular_family_flag']='family non-popular - bulk, pets, alcohol'

In [56]:
cust_prof.loc[(cust_prof['department_id'].isin(regularly_popular))&(cust_prof['number_dependants']>0),'popular_family_flag']='family regular - others'

In [57]:

cust_prof.loc[(cust_prof['department_id'].isin(popular))&(cust_prof['number_dependants']==0),'popular_family_flag']='singles popular - fresh and frozen'

In [58]:
cust_prof.loc[(cust_prof['department_id'].isin(least_popular))&(cust_prof['number_dependants']==0),'popular_family_flag']='singles non-popular - bulk, pets, alcohol'

In [59]:
cust_prof.loc[(cust_prof['department_id'].isin(regularly_popular))&(cust_prof['number_dependants']==0),'popular_family_flag']='singles regular - others'

In [60]:
#Now checking if it worked!
cust_prof['popular_family_flag'].value_counts(dropna=False)

family popular - fresh and frozen            16281549
family regular - others                       6666095
singles popular - fresh and frozen            5434944
singles regular - others                      2211659
family non-popular - bulk, pets, alcohol       277239
singles non-popular - bulk, pets, alcohol       93078
Name: popular_family_flag, dtype: int64

### 0.4 Exporting dataframe  (normal_cust_prods)

In [61]:
cust_prof.to_pickle(os.path.join(path, '02 Data','Prepared data', 'normal_cust_prods.pkl'))